In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from IPython.display import display
# from sklearn.impute import KNNImputer # nie używam
# from sklearn.neighbors import base
import sklearn
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

In [2]:
vaccinationsDF = pd.read_csv('dataFiles/vaccinations.csv', sep=',', header=0, na_filter=False)
casesDF = pd.read_csv('dataFiles/covidCases.csv', sep=',', header=0, na_filter=False)
iso_codesDF = pd.read_csv('dataFiles/iso_codes.csv', sep=',', header=1, na_filter=False)

In [3]:
casesDF.rename(columns={'Date_reported':'date', 'Country':'location', 'Country_code': 'iso2Code'}, inplace=True)

vaccinationsDF.rename(columns={'iso_code': 'iso3Code'}, inplace=True)
vaccinationsDF.drop(vaccinationsDF[(vaccinationsDF['iso3Code'].apply(len) != 3)].index, inplace=True)
vaccinationsDF.reset_index(drop=True)

iso_codesDF.drop(iso_codesDF.columns[[1, 2, 5, 6, 7]], axis=1, inplace=True)
iso_codesDF.rename(columns={'Country name':'location', 'Alpha-2 code':'iso2Code', 'Alpha-3 code':'iso3Code'}, inplace=True)
iso_codesDF = iso_codesDF.astype(str)
iso_codesDF.drop(iso_codesDF[(iso_codesDF['iso2Code'].apply(len) != 2) | (iso_codesDF['iso3Code'].apply(len) != 3)].index, inplace=True)
iso_codesDF = iso_codesDF.reset_index(drop=True)

In [4]:
tmp_df = reduce(lambda left, right: pd.merge(left, right, on=['iso3Code'], how='inner'), [vaccinationsDF, iso_codesDF])
df = reduce(lambda left, right: pd.merge(left, right, on=['iso2Code', 'date'], how='outer'), [tmp_df, casesDF])

In [5]:
df['date'] = pd.to_datetime(df['date'], format='%Y.%m.%d').apply(lambda x: x.toordinal())

In [6]:
df = df.drop(['location_x', 'location_y', 'WHO_region', 'iso3Code', 'total_vaccinations_per_hundred', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred', 'daily_vaccinations_per_million', 'daily_people_vaccinated_per_hundred', 'daily_vaccinations_raw'], axis=1)

In [7]:
df = df.dropna(subset=['location'])
df = df.reset_index(drop=True)

In [ ]:
df_dummies = pd.get_dummies(df)

In [ ]:
df_dummies_columns = df_dummies.columns
df_array = df_dummies.to_numpy()

In [ ]:
RF_transformator = MissForest()
df_array_transformed = RF_transformator.fit_transform(df_array)

In [ ]:
df_transformed = pd.Dataframe(df_array_transformed).transpose()
df_transformed.columns = df_dummies_columns

In [ ]:
display(df_transformed.head(15))

In [ ]:
# df.to_csv('merged.csv', header=True, sep=';')

In [ ]:
"""

# display(casesDF['location'].where(casesDF['iso2Code'].isnull()).unique())
# display(casesDF['iso2Code'].where(casesDF['location'].isnull()).unique())
# display(df.loc[df['iso2Code'] == 'XA'])
# display(df.isnull().sum())
# display(df.count())
# display(df[df['iso3Code'].isna()])
display(df.date)

display(len(df.location_x.unique()))
display(len(df.location_y.unique()))
display(len(df.location.unique()))
# display(casesDF.tail(5))
# display(vaccinationsDF.tail(5))

# display(vaccinationsDF)
display(iso_codesDF.location.unique().size)
display(len(vaccinationsDF.iso3Code.unique()))
# display(df['location'].where(['iso2Code'].isnull()))
display(df.loc[df['location_x'].isnull()])
display(casesDF.count())


def common_member(a, b):
    result = [i for i in a if i not in b]
    return len(result)

# display(len(common_member(casesDF.location.unique(), vaccinationsDF.location.unique())))

# display(common_member(casesDF.iso2Code.unique(), iso_codesDF.iso2Code.unique()))
# display(common_member(vaccinationsDF.iso3Code.unique(), iso_codesDF.iso3Code.unique()))
# display(casesDF.iso2Code.unique())
# display(iso_codesDF.loc[iso_codesDF['location'] == 'Namibia'])
display(common_member(df.location.unique(), df.location_x.unique()))
"""